# Asynchronous GraphQL Data Fetching data Mahasiswa
Notebook ini menunjukkan bagaimana kita bisa melakukan asynchronous data fetching dengan GraphQL SiCekCok untuk mengambil data mahasiswa menggunakan Python. Notebook ini menggunakan library `aiohttp` untuk melakukan HTTP request secara asynchronous dan `aiographql` untuk melakukan query ke GraphQL server.

### Langkah 0: Install Pustaka yang Diperlukan
Pada langkah Persiapan, kita menginstall pustaka `aiohttp` untuk melakukan HTTP request secara asynchronous dan `pandas` untuk memproses data dalam format DataFrame.

In [10]:
%pip install aiohttp pandas

Note: you may need to restart the kernel to use updated packages.


### Langkah 1: Mengimpor Pustaka yang Diperlukan
Pada langkah pertama, kita mengimpor pustaka `aiohttp` `asyncio`  untuk untuk melakukan HTTP request secara asynchronous dan `pandas` untuk memproses data dalam format DataFrame.


In [11]:
import aiohttp
import asyncio
import pandas as pd

### Langkah 2: Mendefinisikan Fungsi untuk Query GraphQL
Kita mendefinisikan fungsi `query_graphql` yang mengambil parameter `nim` dan mengirimkan permintaan POST ke endpoint GraphQL yang ditentukan. Fungsi ini mengembalikan hasil dalam format JSON.

In [12]:
async def query_graphql(session, nim):
    url = 'https://sicekcok.if.unismuh.ac.id/graphql'  # Ganti dengan URL endpoint GraphQL yang sesuai
    query = """
    query($nim: String!) {
      mahasiswa(nim: $nim) {
        nim
        kodeProdi
        angkatan
        nama
        jenisKelamin
        semesterAwal
        tahunAkademikLulus
        tanggalLulus
        lulus
        masaStudi
        khs {
          tahunAkademik
          ips
          sksSmt
          ipk
          sksTotal
          statusMahasiswa
        }
        prodi{
          namaProdi
        }
      }
    }
    """
    variables = {'nim': str(nim)}  # Pastikan NIM dikonversi ke string
    async with session.post(url, json={'query': query, 'variables': variables}) as response:
        return await response.json()

### Langkah 3: Membaca File NIM dan Menyiapkan Daftar Data
Langkah ini membaca file `nim.csv` yang berisi daftar NIM dan menyiapkan struktur data untuk menyimpan hasil unduhan.

In [13]:
nim_list = pd.read_csv('../data/nim.csv')['nim'].tolist()
data_list = []
total = len(nim_list)
max_khs = 0
print(f"Total NIM: {total}")

Total NIM: 6


### Langkah 4: Mengunduh Data secara asynchronousdan Memprosesnya 
Kita mengiterasi daftar NIM, mengunduh data untuk setiap NIM menggunakan fungsi `query_graphql`, dan memproses data tersebut untuk disimpan dalam daftar `data_list`. Juga, kita melacak jumlah maksimum entri KHS untuk mengatur kolom DataFrame nantinya.

In [14]:
async def get_mahasiswa():
    async with aiohttp.ClientSession() as session:
        tasks = []
        for nim in nim_list:
            tasks.append(query_graphql(session, nim))
        responses = await asyncio.gather(*tasks)
        for i, result in enumerate(responses):
            nim = nim_list[i]
            # print(f"Result for NIM {nim}: {result.get('data', {}).get('mahasiswa', {}).get('nama')}")
            mahasiswa = result.get('data', {}).get('mahasiswa', {})
            if mahasiswa:
                row = {
                    'kodeProdi': mahasiswa.get('kodeProdi'),
                    'namaProdi': mahasiswa.get('prodi', {}).get('namaProdi'),
                    'angkatan': mahasiswa.get('angkatan'),
                    'semesterAwal': mahasiswa.get('semesterAwal'),
                    'nim': mahasiswa.get('nim'),
                    'nama': mahasiswa.get('nama'),
                    'jenisKelamin': mahasiswa.get('jenisKelamin'),
                    'tahunAkademikLulus': mahasiswa.get('tahunAkademikLulus'),
                    'tanggalLulus': mahasiswa.get('tanggalLulus'),
                    'lulus': mahasiswa.get('lulus'),
                    'masaStudi': mahasiswa.get('masaStudi'),
                }
                khs = mahasiswa.get('khs', [])
                global max_khs
                max_khs = max(max_khs, len(khs))
                for idx, k in enumerate(khs, start=1):
                    row.update({
                        f'khs{idx}_tahunAkademik': k.get('tahunAkademik'),
                        f'khs{idx}_ips': k.get('ips'),
                        f'khs{idx}_sksSmt': k.get('sksSmt'),
                        f'khs{idx}_ipk': k.get('ipk'),
                        f'khs{idx}_sksTotal': k.get('sksTotal'),
                        f'khs{idx}_statusMahasiswa': k.get('statusMahasiswa'),
                    })
                data_list.append(row)
                # print(f'Downloading data {i + 1}/{total} berhasil di download')
await get_mahasiswa()
print(f'{total} Data berhasil di download')

6 Data berhasil di download


### Langkah 5: Membuat DataFrame dan Menyimpannya ke CSV
Kita membuat DataFrame dari daftar data yang telah diproses.

In [15]:
df = pd.DataFrame(data_list)
columns = [
    'kodeProdi', 'namaProdi', 'angkatan', 'semesterAwal', 'nim', 'nama', 'jenisKelamin', 'tahunAkademikLulus', 'tanggalLulus', 'lulus', 'masaStudi'
] + [
    f'khs{idx}_{field}' for idx in range(1, max_khs + 1) for field in
    ['tahunAkademik', 'ips', 'sksSmt', 'ipk', 'sksTotal', 'statusMahasiswa']
]
df = df.reindex(columns=columns)


### Langkah 6: Memverifikasi Data
Langkah terakhir adalah memverifikasi beberapa baris pertama dari DataFrame untuk memastikan bahwa data telah diunduh dan diproses dengan benar.

In [16]:
pd.set_option('display.max_columns', None)  # Menampilkan semua kolom
pd.set_option('display.expand_frame_repr', False)  # Menghindari pembungkusan frame
df.head(10)

,kodeProdi,namaProdi,angkatan,semesterAwal,nim,nama,jenisKelamin,tahunAkademikLulus,tanggalLulus,lulus,masaStudi,khs1_tahunAkademik,khs1_ips,khs1_sksSmt,khs1_ipk,khs1_sksTotal,khs1_statusMahasiswa,khs2_tahunAkademik,khs2_ips,khs2_sksSmt,khs2_ipk,khs2_sksTotal,khs2_statusMahasiswa,khs3_tahunAkademik,khs3_ips,khs3_sksSmt,khs3_ipk,khs3_sksTotal,khs3_statusMahasiswa,khs4_tahunAkademik,khs4_ips,khs4_sksSmt,khs4_ipk,khs4_sksTotal,khs4_statusMahasiswa,khs5_tahunAkademik,khs5_ips,khs5_sksSmt,khs5_ipk,khs5_sksTotal,khs5_statusMahasiswa,khs6_tahunAkademik,khs6_ips,khs6_sksSmt,khs6_ipk,khs6_sksTotal,khs6_statusMahasiswa,khs7_tahunAkademik,khs7_ips,khs7_sksSmt,khs7_ipk,khs7_sksTotal,khs7_statusMahasiswa,khs8_tahunAkademik,khs8_ips,khs8_sksSmt,khs8_ipk,khs8_sksTotal,khs8_statusMahasiswa,khs9_tahunAkademik,khs9_ips,khs9_sksSmt,khs9_ipk,khs9_sksTotal,khs9_statusMahasiswa,khs10_tahunAkademik,khs10_ips,khs10_sksSmt,khs10_ipk,khs10_sksTotal,khs10_statusMahasiswa,khs11_tahunAkademik,khs11_ips,khs11_sksSmt,khs11_ipk,khs11_sksTotal,khs11_statusMahasiswa,khs12_tahunAkademik,khs12_ips,khs12_sksSmt,khs12_ipk,khs12_sksTotal,khs12_statusMahasiswa,khs13_tahunAkademik,khs13_ips,khs13_sksSmt,khs13_ipk,khs13_sksTotal,khs13_statusMahasiswa,khs14_tahunAkademik,khs14_ips,khs14_sksSmt,khs14_ipk,khs14_sksTotal,khs14_statusMahasiswa
0,22202,Pengairan,2015,20151,10581232015,MUAYYIN NAZAR,L,20191,2020-02-08,True,"4 Tahun, 5 Bulan",20151,3.32,22,3.45,20,A,20152,2.77,22,3.24,37,A,20161,2.73,22,3.20,55,A,20162,2.83,24,3.21,73,A,20171,2.67,24,3.23,91,A,20172,3.20,20,3.25,109,A,20181,3.33,18,3.26,127,A,20182,3.90,20.0,3.35,147.0,A,20191,4.00,8.0,3.38,155.0,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,22202,Pengairan,2015,20151,10581232115,RIDWAN,L,None,None,False,None,20151,2.82,22,3.10,20,A,20152,3.09,22,3.10,42,A,20161,2.00,22,2.90,60,A,20162,2.60,20,2.90,78,A,20171,1.20,20,2.91,86,A,20182,0.00,0,2.91,86,N,20231,0.00,0,2.91,86,X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,23201,Arsitektur,2018,20181,105831107018,AHMAD FADLI,L,20232,2024-08-30,True,"5 Tahun, 11 Bulan",20181,3.38,21,3.38,21,A,20182,2.65,23,3.30,40,A,20191,2.55,20,3.21,57,A,20192,1.90,20,3.16,70,A,20201,2.82,17,3.09,87,A,20202,2.20,20,3.13,100,A,20211,3.35,20,3.23,115,A,20212,2.63,23.0,3.33,128.0,A,20221,1.17,12.0,3.34,132.0,A,20222,2.60,15.0,3.35,143.0,A,20231,0.57,14.0,3.36,145.0,A,20232,3.92,12.0,3.40,157.0,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,55202,Informatika,2018,20181,105841100118,MUNAWAR PUJA DEWANTARA,L,None,None,False,None,20181,2.05,22,2.50,18,A,20182,1.00,23,2.19,31,A,20191,0.00,0,2.19,31,N,20192,0.00,0,2.19,31,N,20201,0.00,0,2.19,31,N,20202,0.00,0,2.19,31,N,20211,0.00,0,2.19,31,N,20212,0.00,0.0,2.19,31.0,N,20221,0.00,0.0,2.19,31.0,N,20222,0.00,0.0,2.19,31.0,N,20231,0.00,0.0,2.19,31.0,N,20232,0.00,0.0,2.19,31.0,N,20241,0.00,0.0,2.19,31.0,N,20242,0.00,0.0,2.19,31.0,N
4,55202,Informatika,2018,20181,105841100218,ADIWIRATMAN,L,20242,2025-08-30,True,"6 Tahun, 11 Bulan",20181,2.64,22,2.64,22,A,20182,2.43,23,2.53,45,A,20191,1.18,22,2.50,56,A,20192,1.24,21,2.41,69,A,20201,0.75,16,2.31,77,A,20202,3.29,14,2.46,91,A,20211,3.10,24,2.64,113,A,20212,1.89,23.0,2.68,126.0,A,20221,1.47,17.0,2.70,133.0,A,20222,0.36,11.0,2.69,135.0,A,20231,1.27,11.0,2.67,141.0,A,20232,1.50,9.0,2.69,145.0,A,20241,0.62,13.0,2.71,147.0,A,20242,3.38,13.0,2.82,156.0,A
5,55202,Informatika,2018,20181,105841100318,MUH. ICHZAN,L,None,None,False,None,20181,0.00,22,0.00,0,A,20182,0.00,0,0.00,0,N,20191,0.00,0,0.00,0,N,20192,0.00,0,0.00,0,N,20201,0.00,0,0.00,0,N,20202,0.00,0,0.00,0,N,20211,0.00,0,0.00,0,N,20212,0.00,0.0,0.00,0.0,N,20221,0.00,0.0,0.00,0.0,N,20222,0.00,0.0,0.00,0.0,N,20231,0.00,0.0,0.00,0.0,X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Validasi data yang sudah di download dengan NIM yang dicari

In [17]:
# Validasi data yang sudah di download dengan NIM yang dicari
nim_yang_dicari = '105821100817'  # Ganti dengan NIM yang ingin Anda cari
data_nim = df.loc[df['nim'] == nim_yang_dicari]

# Menampilkan hasil pencarian dan cocokkan dengan KHS yang ada
if not data_nim.empty:
    for index, row in data_nim.iterrows():
        print(f"Data Mahasiswa dengan NIM: {row['nim']}")
        print("=================================")
        for column in data_nim.columns:
            print(f"{column}: {row[column]}")
else:
    print("Data dengan NIM tersebut tidak ditemukan.")

Data dengan NIM tersebut tidak ditemukan.


# Simpan Ke File .csv

In [18]:
from datetime import datetime

# Generate the timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Create the filename with the timestamp
filename = f"../outputs/mahasiswa_data_{timestamp}.csv"

# Save the DataFrame to the CSV file
df.to_csv(filename, index=False)

# Print the success message with the filename
print(f"Data berhasil disimpan ke file {filename}")

Data berhasil disimpan ke file ../outputs/mahasiswa_data_20251005_063448.csv
